In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import json
import re
import numpy as np
import pandas as pd

# Evaluation datasets




In [2]:
with open("./data/datasets/python_full.json", "r") as f:
    df_python_full = pd.read_json(f, lines=True)

In [3]:
df_python_full.head()

,signature,body,docstring,id
0,"def debug(user, message):","message_user(user, message, constants.DEBUG)<EOL>",Adds a message with the ``DEBUG`` level.\n\n:p...,f4:m0
1,"def success(user, message):","message_user(user, message, constants.SUCCESS)...",Adds a message with the ``SUCCESS`` level.\n\n...,f4:m2
2,"def process_response(self, request, response):","if hasattr(request, ""<STR_LIT>"") and hasattr(r...","Check for messages for this user and, if it ex...",f5:c0:m0
3,"def table(name, auth=None, eager=True):",auth = auth or []<EOL>dynamodb = boto.connect_...,Returns a given table for the given user.,f8:m0
4,"def api_request(methods=None, require_token=Tr...",def decorator(view_func):<EOL><INDENT>@wraps(v...,View decorator that handles JSON based API req...,f21:m0


In [4]:
len(df_python_full)

20000

In [5]:
with open("./data/datasets/kotlin_full.csv", "r") as f:
    df_kotlin_full = pd.read_csv(f)

In [6]:
df_kotlin_full

,signature,body,docstring,id
0,"fun <S> intersection(x: Inv<in S>, y: Inv<in S>)",: S = T,NaN,356b
1,No signature found,"fun <K> use(k: K, f: K.(K) -> K) {}",NaN,6e1f
2,No signature found,"fun <K> useNested(k: K, f: Inv<K>.(Inv<K>) -> ...",NaN,6e1f
3,No signature found,fun <T> createDelegate(f: () -> T): Delegate<T...,NaN,6e1f
4,No signature found,"fun getValue(thisRef: Any?, property: kotlin.r...",NaN,6e1f
...,...,...,...,...
10000,fun containsKey(key: Z<Int>),: Boolean = k,NaN,6107
10001,fun containsValue(value: Z<Int>),: Boolean = s,NaN,a991
10002,fun get(key: Z<Int>),= s,NaN,cd8c
10003,fun isEmpty(),: Boolean = s,NaN,464e


In [7]:
len(df_kotlin_full)

10005

In [8]:
print(df_kotlin_full.info())
print(df_kotlin_full.isnull().sum())

unique_signatures = df_kotlin_full['signature'].nunique()
print(f"Number of unique function signatures: {unique_signatures}")

most_common_ids = df_kotlin_full['id'].value_counts().head(10)
print("Most common identifiers:")
print(most_common_ids)

with_docstrings = df_kotlin_full['docstring'].apply(lambda x: not pd.isna(x) and x != 'NaN').sum()
without_docstrings = df_kotlin_full['docstring'].size - with_docstrings
print(f"Functions with docstrings: {with_docstrings}")
print(f"Functions without docstrings: {without_docstrings}")

no_signature = df_kotlin_full[df_kotlin_full['signature'] == 'No signature found']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10005 entries, 0 to 10004
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   signature  10005 non-null  object
 1   body       10005 non-null  object
 2   docstring  1324 non-null   object
 3   id         10005 non-null  object
dtypes: object(4)
memory usage: 312.8+ KB
None
signature       0
body            0
docstring    8681
id              0
dtype: int64
Number of unique function signatures: 4221
Most common identifiers:
id
6e1f    3181
4f27     586
519f     366
fe98     163
66c3      95
f6fb      58
a535      54
d987      51
6874      26
b22f      23
Name: count, dtype: int64
Functions with docstrings: 1324
Functions without docstrings: 8681


In [9]:
df_unique = df_kotlin_full.drop_duplicates()

print(f"Original dataset size: {df_kotlin_full.shape}")
print(f"Cleaned dataset size: {df_unique.shape}")

Original dataset size: (10005, 4)
Cleaned dataset size: (9422, 4)


In [10]:
filtered_df = df_unique[
    (df_unique['docstring'].notna()) &
    (df_unique['docstring'] != 'NaN') &
    (df_unique['signature'] != 'No signature found')
]

test_dataset = filtered_df.sample(n=100, random_state=1)

Test dataset saved with 100 rows.


In [11]:
def extract_signature_from_body(row):
    if row['signature'] == 'No signature found':
        match = re.search(r'fun\s+<[^>]*>\s+[\w]+<[^>]*>\s*\(.*?\)', row['body'])
        if match:
            return match.group(0)
        else:
            return 'Signature extraction failed'
    else:
        return row['signature']

filtered_df['signature'] = filtered_df.apply(extract_signature_from_body, axis=1)

filtered_df.to_csv('./data/datasets/kotlin_full.csv', index=False)

print("Updated dataset saved.")

Updated dataset saved.


/var/folders/72/j58pmsws2xgc0695c97h4q280000gn/T/ipykernel_80484/2502014185.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['signature'] = filtered_df.apply(extract_signature_from_body, axis=1)


In [ ]:
# test_dataset.to_csv('./data/datasets/kotlin_test.csv', index=False)

# print(f"Test dataset saved with {test_dataset.shape[0]} rows.")